# Setup CUDA

In [15]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [16]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-028bleof
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-028bleof
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done


In [17]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


#CUDA Parallelism Code

## Kode Percobaan Buat Test Setup Cuda (Bukan Buat Tucil CUDA)

In [18]:
%%cu
#include <iostream>
    int main() {
      std::cout << "Welcome To GeeksforGeeks\n";
      return 0;
    }

Welcome To GeeksforGeeks



In [19]:
%%cu
#include <cstdio>
#include <iostream>

	using namespace std;

__global__ void maxi(int* a, int* b, int n)
{
	int block = 256 * blockIdx.x;
	int max = 0;

	for (int i = block; i < min(256 + block, n); i++) {

		if (max < a[i]) {
			max = a[i];
		}
	}
	b[blockIdx.x] = max;
}

int main()
{

	int n;
	n = 3 >> 2;
	int a[n];

	for (int i = 0; i < n; i++) {
		a[i] = rand() % n;
		cout << a[i] << "\t";
	}

	cudaEvent_t start, end;
	int *ad, *bd;
	int size = n * sizeof(int);
	cudaMalloc(&ad, size);
	cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
	int grids = ceil(n * 1.0f / 256.0f);
	cudaMalloc(&bd, grids * sizeof(int));

	dim3 grid(grids, 1);
	dim3 block(1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while (n > 1) {
		maxi<<<grids, block>>>(ad, bd, n);
		n = ceil(n * 1.0f / 256.0f);
		cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time = 0;
	cudaEventElapsedTime(&time, start, end);

	int ans[2];
	cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);

	cout << "The maximum element is : " << ans[0] << endl;

	cout << "The time required : ";
	cout << time << endl;
}


The maximum element is : 0
The time required : 0.003904



## Kode Cuda Buat Tucil

###Cara Run :
- upload "cuda.cu" yang telah kita buat
- upload test_case.txt (kalo udah butuh test case) => ini buat tucil
- jalankan command
- testing sample => !time ./cuda
- testing tucil => !time ./cuda < ./test_case/128.txt > output_cuda_128.txt

In [20]:
!nvcc cuda.cu -o cuda

In [21]:
# Command Buat Jalanin test sample cuda
!time ./cuda

# Command Buat Tucil Cuda (ini masih dugaan sih)
# !time ./cuda < ./test_case/128.txt > output_cuda_128.txt

Hello world from x.0 y.0 z.0!
Hello world from x.1 y.0 z.0!
Hello world from x.0 y.1 z.0!
Hello world from x.1 y.1 z.0!
Hello world from x.0 y.0 z.1!
Hello world from x.1 y.0 z.1!
Hello world from x.0 y.1 z.1!
Hello world from x.1 y.1 z.1!
Hello world from x.0 y.0 z.0!
Hello world from x.1 y.0 z.0!
Hello world from x.0 y.1 z.0!
Hello world from x.1 y.1 z.0!
Hello world from x.0 y.0 z.1!
Hello world from x.1 y.0 z.1!
Hello world from x.0 y.1 z.1!
Hello world from x.1 y.1 z.1!
Hello world from x.0 y.0 z.0!
Hello world from x.1 y.0 z.0!
Hello world from x.0 y.1 z.0!
Hello world from x.1 y.1 z.0!
Hello world from x.0 y.0 z.1!
Hello world from x.1 y.0 z.1!
Hello world from x.0 y.1 z.1!
Hello world from x.1 y.1 z.1!
Hello world from x.0 y.0 z.0!
Hello world from x.1 y.0 z.0!
Hello world from x.0 y.1 z.0!
Hello world from x.1 y.1 z.0!
Hello world from x.0 y.0 z.1!
Hello world from x.1 y.0 z.1!
Hello world from x.0 y.1 z.1!
Hello world from x.1 y.1 z.1!

real	0m0.850s
user	0m0.015s
sys	0m0.748